In [19]:
#@title import dependencies
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
import joblib
import numpy as np
import random

# Set seeds for reproducibility
SEED = 41
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# Machine Unlearning on a MNIST classifier - BASELINE test

##Setup and Initial Training
To begin, we'll train a neural network to classify the MNIST dataset. This section covers the steps from setting up environment to evaluating the model's accuracy. If you prefer, you can skip this section and use the pre-trained model from the GitHub repository (instructions for downloading are in the next section).

In [20]:
#@title loading and preprocessing

# Define transformations for the training and validation data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

In [21]:
#@title define the neural network

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28) # Flatten the image
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = NN()

In [22]:
#@title define loss function and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [23]:
#@title train the network

n_epochs = 10
for epoch in tqdm(range(n_epochs)):
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f' Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}')

 10%|█         | 1/10 [00:19<02:59, 19.97s/it]

 Epoch 1, Loss: 1.0238848064881143


 20%|██        | 2/10 [00:37<02:27, 18.38s/it]

 Epoch 2, Loss: 0.38597884826632195


 30%|███       | 3/10 [00:54<02:06, 18.03s/it]

...

 Epoch 8, Loss: 0.20669279496139809


 90%|█████████ | 9/10 [02:44<00:18, 18.12s/it]

 Epoch 9, Loss: 0.19195180559066186


100%|██████████| 10/10 [03:01<00:00, 18.19s/it]

 Epoch 10, Loss: 0.17762162606678664


In [24]:
#@title evaluate the network

correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {100 * correct / total}%')

Accuracy on the test set: 94.9%


In [26]:
#@title download the model

joblib.dump(model, "MNIST_Classifier.joblib")

['MNIST_Classifier.joblib']